In [5]:
""" This notebook used for preporation data to NLP modeling.
 Proceses of Tokenization, Stemming, Lemmatization, Handling text (Remove HTML Tag, URLs, Emojies and other) are here.   """

import pandas as pd
import re                                              # Import Regular Expression (remove HTML tags)
import string                                          # Import Punctuation 
from textblob import TextBlob                          # Import this Library to Handle the Spelling Issue
import nltk
from nltk.corpus import stopwords                      #  NLTK library to remove Stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import emoji                                           # for translating symbol to text
import spacy                                           # for tokenization
import spacy.cli
# spacy.cli.download("en_core_web_sm")                 # for  working with spacy, after the first start should pick  # spacy.cli.download("en_core_web_lg")
from nltk.stem.porter import PorterStemmer             # for stemming
# nltk.download('all')                                   # for  working with NLTL function, after the first start should pick #nltk.download('all') 
from sklearn.model_selection import train_test_split
from chat_words import chat_word                       # for translate slang of charts to text
from autocorrect import Speller                        # for Spelling Correction
from collections import Counter, OrderedDict                        # for definition of unique words (tokens) in dataframe
from torchtext.vocab import vocab  
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'chat_words'

In [2]:
df = pd.read_csv(r'C:\Users\Admin\WORK\Project_CV\Model_NLP_sentiment\data\IMDB Dataset.csv')    # insert path to your data

In [3]:
df.head(2)                                                # check dataframe 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [ ]:
# # transform sentiment  into the number labels
# def transform_label(label):
#     return 1 if label == 'positive' else 0

# df['label'] = df['sentiment'].apply(transform_label)
# df.head(2)

In [ ]:
# #Access the corpus and target variables
# x = df.review
# y = df.label                                                                            

# # train test splitting
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.0005, random_state=0)
# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(y_test.shape)


In [6]:
# Choose items for preprocessing: True or False

lower = True                           # LoweCasing Text
remove_html = True                     # Remove HTML Tag
remove_url = True                       # Remove URLs
remove_punc = True                     # Remove punctuation
change_chat = True                     # Handling chat's words to words
spell_cor = True                       # Spelling Correction
remove_stopword = True                 # Remove StopWords
remove_emoji = True                  # Handling Emojies to words
use_stemm = False                       # Apply Stemming
use_lemm = True                        # Apply Lemmatization
use_token = True                        # Apply Tokenization  

In [11]:
# Function for preprocessing
def preprocessing (text):
        
    if lower:                                                        # LoweCasing Text
        text = text.lower()
                                            
    if remove_html:
        pattern_1 = re.compile('<.*?>')                              # constant using one regular expression
        text = re.sub(pattern_1, r'', text)                          # Remove HTML Tags (changes ('<.*?>') to gap " ")

    if remove_url:
        pattern_2 = re.compile(r'https?://\S+|www\.\S+')             #  Remove URLs from Text or Whole Corpus.
        text = pattern_2.sub(r'', text)

    if remove_punc:
        punc = string.punctuation                                    # Remove punctuation
        text = text.translate(str.maketrans('', '', punc))

    if change_chat:
        new_text = []                                                 # changes chat's words to text       
        for i in text.split():
            if i.upper() in chat_word:
                new_text.append(chat_word[i.upper()])
            else:
                new_text.append(i)
        text = " ".join(new_text)
        new_text.clear()

    if spell_cor:
        spell = Speller(lang='en')                                    # Spelling Correction
        text = spell(text)

    if remove_stopword:
        stopword = stopwords.words('english')                          # Handling StopWords
        for word in text.split():
            if word in stopword:
                new_text.append('')
            else:
                new_text.append(word)
        pattern_3 = new_text[:]
        text = " ".join(pattern_3)

    if remove_emoji:
        text = emoji.demojize(text)                                   # Handling Emojies 

    
    if use_stemm:
        stemmer = PorterStemmer()                                     # Stemming
        text = " ".join([stemmer.stem(word)
                  for word in text.split()])
                            
        
    if use_lemm:
        lemmatizer = WordNetLemmatizer()                              #Lemmatization
        words = nltk.word_tokenize(text)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        text = ' '.join(lemmatized_words)


    if use_token:
        nlp = spacy.load('en_core_web_sm')                            # the English language model 'en_core_web_sm'
        text = nlp(text)                                              
    return text

In [1]:
# # Applying function for preprocessing

# # X_trainn = X_train.apply(preprocessing)
# x_testt = x_test.apply(preprocessing)
# x_testt.head(2)
df['review'] = df['review'].apply(preprocessing)

NameError: name 'df' is not defined

In [13]:
# This block for checking  def Proprocessing

check_data = {
    "review": ["@lapcat need to send 'em to my accountant tomorrow. oddly, i wasn't even referring to my taxes. those are supporting evidence, though. ",
                "<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href='http://google.com'>download</a></p></body></html>",
                 'Check out my notebook https://www.kaggle.com/campusx/notebook8223fc1', 'IMHO he is the best', 'FYI Islamabad is the capital of Pakistan',
                 'ceertain conditionas duriing seveal ggenerations aree moodified in the saame maner', 'probably my all-time favorite movie, a story of selflessness,'
                 ' sacrifice and dedication to a noble cause', "Loved the movie. It was 😘", "walk walks walking walked",
                 "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."
]
    }

# Convert to DataFrame
df_check = pd.DataFrame(check_data)

# Print the DataFrame
prov = df_check['review']
prov

0    @lapcat need to send 'em to my accountant tomo...
1    <html><body><p> Movie 1</p><p> Actor - Aamir K...
2    Check out my notebook https://www.kaggle.com/c...
3                                  IMHO he is the best
4             FYI Islamabad is the capital of Pakistan
5    ceertain conditionas duriing seveal ggeneratio...
6    probably my all-time favorite movie, a story o...
7                            Loved the movie. It was 😘
8                            walk walks walking walked
9    He was running and eating at same time. He has...
Name: review, dtype: object

In [14]:
prov_check = prov.apply(preprocessing)
prov_check

0    (lancet, need, send, em, accountant, tomorrow,...
1       (movie, 1, actor, amir, khan, click, download)
2                                    (check, notebook)
3           (In, My, Honest, /, Humble, Opinion, best)
4    (For, Your, Information, islamabad, capital, p...
5    (certain, condition, several, generation, modi...
6    (probably, alltime, favorite, movie, story, se...
7            (loved, movie, :, face_blowing_a_kiss, :)
8                        (walk, walk, walking, walked)
9    (running, eating, Tears, eye, bad, habit, swim...
Name: review, dtype: object

In [15]:
# get all processed r
# Definition of unique words (tokens) in dataframe

token_counts = Counter()
for word in x_testt:
    token_counts.update(word)



In [16]:
print('Dictionary size:', len(token_counts))

Dictionary size: 2520


In [ ]:


sorted_by_freg_tuples = sorted (token_counts.items(), key=lambda x: x[1], reverse = True)
ordered_dict = OrderedDict(sorted_by_freg_tuples)
vocab = vocab(ordered_dict)
vocab.insert_token('<pad>', 0)
vocab.insert_token('<unk>', 1)
vocab.set_default_index(1)
# print([vocab[token] for token in ['Hot','run'] ])

In [30]:
print(type(x_testt))
print(type(df))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [28]:
all_text2 = x_testt['review'].tolist()

# get all processed reviews

# merge into single variable, separated by whitespaces
all_text2 = ' '.join(all_text2)     
# obtain list of words
words = all_text2.split()

# check our list
words[:10]

KeyError: 'review'

In [ ]:
VOCABULARY_SIZE = 20000
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')